First, we import the the training data sets

training_set_labels and training_set_features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.impute import KNNImputer#, IterativeImputer
# from sklearn import svm
from sklearn.model_selection import GridSearchCV

# %mathplotlib inline

# displays all the columns
pd.set_option('display.max_columns', None)
# plt.rcParams["figure.figsize"] = (18, 8);

dataset_features= pd.read_csv('All Datasets/training_set_features.csv')
dataset_labels=pd.read_csv('All Datasets/training_set_labels.csv')

print(dataset_features.shape)
print(dataset_features.columns)

print(dataset_labels.shape)
print(dataset_labels.columns)



(26707, 36)
Index(['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')
(26707, 3)
Index(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], dtype='object')


We have to pre-process the data to convert categorical data to ordinal data, and also fill in any missing data with the appropriate values (Mode for now)
First, let us find the unique catagories in all the columns

In [2]:
for col in dataset_features.columns:
    print(col, dataset_features[col].unique())

respondent_id [    0     1     2 ... 26704 26705 26706]
xyz_concern [ 1.  3.  2.  0. nan]
xyz_knowledge [ 0.  2.  1. nan]
behavioral_antiviral_meds [ 0.  1. nan]
behavioral_avoidance [ 0.  1. nan]
behavioral_face_mask [ 0.  1. nan]
behavioral_wash_hands [ 0.  1. nan]
behavioral_large_gatherings [ 0.  1. nan]
behavioral_outside_home [ 1.  0. nan]
behavioral_touch_face [ 1.  0. nan]
doctor_recc_xyz [ 0. nan  1.]
doctor_recc_seasonal [ 0. nan  1.]
chronic_med_condition [ 0.  1. nan]
child_under_6_months [ 0.  1. nan]
health_worker [ 0.  1. nan]
health_insurance [ 1. nan  0.]
opinion_xyz_vacc_effective [ 3.  5.  4.  2.  1. nan]
opinion_xyz_risk [ 1.  4.  3.  2.  5. nan]
opinion_xyz_sick_from_vacc [ 2.  4.  1.  5.  3. nan]
opinion_seas_vacc_effective [ 2.  4.  5.  3.  1. nan]
opinion_seas_risk [ 1.  2.  4.  3.  5. nan]
opinion_seas_sick_from_vacc [ 2.  4.  1.  5. nan  3.]
age_group ['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
education ['< 12 Years' '12 Yea

In [3]:
mapping_age_group = {"18 - 34 Years": 1, "35 - 44 Years": 2, "45 - 54 Years": 3, "55 - 64 Years": 4, "65+ Years": 5}
mapping_race = {"White": 1, "Black": 2, "Hispanic": 3, "Other or Multiple": 4}
mapping_sex = {"Male": 1, "Female": 2}
mapping_income_poverty = {"Below Poverty": 1, "<= $75,000, Above Poverty": 2, "> $75,000": 3}
mapping_marital_status = {"Married": 1, "Not Married": 2}
mapping_rent_or_own = {"Own": 1, "Rent": 2}
mapping_employment_status = {"Employed": 1, "Not in Labor Force": 2, "Unemployed": 3}
mapping_census_msa = {"Non-MSA": 1, "MSA, Not Principle City": 2, "MSA, Principle City": 3}
mapping_household_adults = {0.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, np.nan: -1}
mapping_household_children = {0.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, np.nan: -1}
mapping_geo = {
    'oxchjgsf': 1,
    'bhuqouqj': 2,
    'qufhixun': 3,
    'lrircsnp': 4,
    'atmpeygn': 5,
    'lzgpxyit': 6,
    'fpwskwrf': 7,
    'mlyzmhmf': 8,
    'dqpwygqj': 9,
    'kbazzjca': 10
}
mapping_employment_industry = {
    # np.nan: -1,
    'pxcmvdjn': 1,
    'rucpziij': 2,
    'wxleyezf': 3,
    'saaquncn': 4,
    'xicduogh': 5,
    'ldnlellj': 6,
    'wlfvacwt': 7,
    'nduyfdeo': 8,
    'fcxhlnwr': 9,
    'vjjrobsf': 10,
    'arjwrbjb': 11,
    'atmlpfrs': 12,
    'msuufmds': 13,
    'xqicxuve': 14,
    'phxvnwax': 15,
    'dotnnunm': 16,
    'mfikgejo': 17,
    'cfqqtusy': 18,
    'mcubkhph': 19,
    'haxffmxo': 20,
    'qnlwzans': 21
}
mapping_employment_occupation = {
    # np.nan: -1,
    'xgwztkwe': 1,
    'xtkaffoo': 2,
    'emcorrxb': 3,
    'vlluhbov': 4,
    'xqwwgdyp': 5,
    'ccgxvspp': 6,
    'qxajmpny': 7,
    'kldqjyjy': 8,
    'mxkfnird': 9,
    'hfxkjkmi': 10,
    'bxpfxfdn': 11,
    'ukymxvdu': 12,
    'cmhcxjea': 13,
    'haliazsg': 14,
    'dlvbwzss': 15,
    'xzmlyyjv': 16,
    'oijqvulv': 17,
    'rcertsgn': 18,
    'tfqavkke': 19,
    'hodpvpew': 20,
    'uqqtjvyb': 21,
    'pvmttkik': 22,
    'dcjcmpih': 23
}
mapping_education = {
    '< 12 Years': 1,
    '12 Years': 2,
    'College Graduate': 3,
    'Some College': 4
}




def cat_to_ord(dataset):
    dataset["age_group"]=dataset.age_group.map(mapping_age_group)
    dataset["race"]=dataset.race.map(mapping_race)
    dataset["sex"]=dataset.sex.map(mapping_sex)
    dataset["income_poverty"] = dataset["income_poverty"].map(mapping_income_poverty)
    dataset["marital_status"] = dataset["marital_status"].map(mapping_marital_status)
    dataset["rent_or_own"] = dataset["rent_or_own"].map(mapping_rent_or_own)
    dataset["employment_status"] = dataset["employment_status"].map(mapping_employment_status)
    dataset["census_msa"] = dataset["census_msa"].map(mapping_census_msa)
    dataset["household_adults"] = dataset["household_adults"].map(mapping_household_adults)
    dataset["household_children"] = dataset["household_children"].map(mapping_household_children)
    dataset["hhs_geo_region"] = dataset["hhs_geo_region"].map(mapping_geo)
    dataset["employment_industry"] = dataset["employment_industry"].map(mapping_employment_industry)
    dataset["employment_occupation"] = dataset["employment_occupation"].map(mapping_employment_occupation)
    dataset["education"] = dataset["education"].map(mapping_education)
    
    return dataset

cat_to_ord(dataset_features)
   
 



print(dataset_features.columns)
# print(dataset.head)
for col in dataset_features.columns:
    print(col, dataset_features[col].unique())


Index(['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')
respondent_id [    0     1     2 ... 26704 26705 26706]
xyz_concern [ 1.  3.  2.  0. nan]
xyz_knowledge [ 0.  2.  1. nan]
beh

Handling Null values with necessary method

In [4]:

def null_fill(dataset):
    for column in dataset.columns:
    #     selected_value = dataset[column].mode()[0]
    #     # selected_value=dataset[column].mean()
        selected_value=-1
        dataset[column].fillna(selected_value, inplace=True)


# imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
# imputer.fit(dataset)
# dataset=imputer.transform(dataset)

# imputer=IterativeImputer()
# imputer.fit(dataset)
# dataset=imputer.transform(dataset)



# print(dataset.head)
null_fill(dataset_features)
print(dataset_features.isnull().sum())
# print('Missing: %d' % sum(np.isnan(dataset).flatten()))

dataset_features=dataset_features

respondent_id                  0
xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_re

We can now split the dataset into training and testing sets. We will use 80% of the data for training and 20% for testing.

In [5]:
y1=dataset_labels['xyz_vaccine']
y2=dataset_labels['seasonal_vaccine']
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(dataset_features, y1, y2, test_size=0.2, random_state=30)

Standardize the Features (if necessary)

Standardizing can be beneficial for logistic regression


In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


We train and optimize both the model for best performance
We can also see the roc score for the models

We can tune the hyperparameters to get better performance

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from scipy.stats import randint
import numpy as np

# Define the parameter distributions
param_dist = {
    'n_estimators': randint(100, 300),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None] + list(np.arange(10, 31, 10)),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5),
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Function to perform randomized search and print results
def perform_randomized_search(X_train, y_train):
    random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1, random_state=42)
    random_search.fit(X_train, y_train)
    
    print(f"Best parameters: {random_search.best_params_}")
    print(f"Best ROC AUC: {random_search.best_score_}")
    
    return random_search.best_estimator_

# Perform randomized search for target_var1
print("Randomized Search for target_var1")
best_rf_model_target_var1 = perform_randomized_search(X_train, y1_train)

# Perform randomized search for target_var2
print("Randomized Search for target_var2")
best_rf_model_target_var2 = perform_randomized_search(X_train, y2_train)

# Evaluate the initial and optimized models on the training data
def evaluate_model(model, X_train, y_train, X_test, y_test, target_name):
    train_pred = model.predict_proba(X_train)[:, 1]
    test_pred = model.predict_proba(X_test)[:, 1]
    
    train_roc_auc = roc_auc_score(y_train, train_pred)
    test_roc_auc = roc_auc_score(y_test, test_pred)
    
    print(f"{target_name} - Train ROC AUC: {train_roc_auc:.4f}")
    print(f"{target_name} - Test ROC AUC: {test_roc_auc:.4f}")

# Initial Random Forest model (default hyperparameters)
initial_rf_model = RandomForestClassifier(random_state=42)
initial_rf_model.fit(X_train, y1_train)
print("\nInitial Random Forest Model for target_var1")
evaluate_model(initial_rf_model, X_train, y1_train, X_test, y1_test, "Initial RF target_var1")

initial_rf_model.fit(X_train, y2_train)
print("\nInitial Random Forest Model for target_var2")
evaluate_model(initial_rf_model, X_train, y2_train, X_test, y2_test, "Initial RF target_var2")

# Evaluate the optimized Random Forest models
print("\nOptimized Random Forest Model for target_var1")
evaluate_model(best_rf_model_target_var1, X_train, y1_train, X_test, y1_test, "Optimized RF target_var1")

print("\nOptimized Random Forest Model for target_var2")
evaluate_model(best_rf_model_target_var2, X_train, y2_train, X_test, y2_test, "Optimized RF target_var2")


Randomized Search for target_var1
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 288}
Best ROC AUC: 0.8617397936873662
Randomized Search for target_var2
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 279}
Best ROC AUC: 0.8564968837875566

Initial Random Forest Model for target_var1
Initial RF target_var1 - Train ROC AUC: 1.0000
Initial RF target_var1 - Test ROC AUC: 0.8635

Initial Random Forest Model for target_var2
Initial RF target_var2 - Train ROC AUC: 1.0000
Initial RF target_var2 - Test ROC AUC: 0.8590

Optimized Random Forest Model for target_var1
Optimized RF target_var1 - Train ROC AUC: 0.9838
Optimized RF target_var1 - Test ROC AUC: 0.8692

Optimized Random Forest M

We now import the test data set and the submition format

We also run the same preproccessing on them

In [8]:
test_features=pd.read_csv("All Datasets/test_set_features.csv")
output=pd.read_csv("All Datasets/submission_format.csv")

print(test_features.shape)
print(test_features.columns)

print(output.shape)
print(output.columns)

ids=test_features["respondent_id"]
# print(output["h1n1_vaccine"])
# print(output["seasonal_vaccine"])

(26708, 36)
Index(['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')
(26708, 3)
Index(['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine'], dtype='object')


In [9]:
cat_to_ord(test_features)
null_fill(test_features)

test_features_np = test_features.values

# Make predictions
output1_prediction = best_rf_model_target_var1.predict(test_features_np)
output2_prediction = best_rf_model_target_var2.predict(test_features_np)

# Add the ids

output["respondent_id"]=ids
output["h1n1_vaccine"]=output1_prediction
output["seasonal_vaccine"]=output2_prediction

output.to_csv('output_final.csv',header=True, index=False)


In [10]:

from playsound import playsound
playsound(r'/Users/VA/Music/Music/Media.localized/Music/Akiyuki Tateyama/TV anime _Yurukyan△_original sound track/01 Kyanpujou no theme motosuko.mp3')

KeyboardInterrupt: 